## Data Pre-processing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
X_df = dataset.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = dataset.iloc[:, 13].values

In [3]:
X_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

## Transform Categorical Variables

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder_gender = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=False)
geography_df = pd.DataFrame(one_hot_encoder.fit_transform(X_df['Geography'].values.reshape(-1, 1)), columns = one_hot_encoder.categories_)
geography_df.drop([geography_df.columns[0]], axis=1, inplace=True)
X_df['Gender'] = label_encoder_gender.fit_transform(X_df['Gender'])
label_gender_mapping = dict(zip(label_encoder_gender.transform(label_encoder_gender.classes_), label_encoder_gender.classes_))
X_df.rename(columns={'Gender': label_gender_mapping[1]}, inplace=True)
X_df = X_df.merge(geography_df, left_index=True, right_index=True)
X_df.drop(['Geography'], axis=1, inplace=True)
'''
only 2 dummy variables for the country: for example spain, france while not labeling germany 
since it can correspond to value zero from both spain and france
'''
#countries_data
X_df.head()

,CreditScore,Male,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,"(Germany,)","(Spain,)"
0,619,0,42,2,0.00,1,1,1,101348.88,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,1.0


## Split Train and Test set

In [6]:
from sklearn.model_selection import train_test_split
X = X_df.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Artificial Neural Network Implementation

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [9]:
def build_classifier(optimizer='adam', nn_count = 6):
    #Initializing the ANN
    classifier = Sequential() #Define as a sequence of layers

    #add input layer and first hidden layer with dropout
    #randomly initialize weights close to zero (using kernel initializer parameter)
    #using 6 as output dimension as this is the average between input layers + output layers
    input_to_hidden_layer = Dense(units = nn_count, kernel_initializer="uniform", activation = 'relu', input_dim = 11)
    classifier.add(input_to_hidden_layer)
    #disable random neurons at every iteration to lower overfitting
    classifier.add(Dropout(rate = 0.1)) #disable 10% of the neurons at every iteration 
    
    #add second hidden layer
    second_hidden_layer = Dense(units = nn_count, kernel_initializer="uniform", activation = 'relu')
    classifier.add(second_hidden_layer)
    classifier.add(Dropout(rate = 0.1)) #disable 10% of the neurons at every iteration 

    #add output layer
    output_layer = Dense(units = 1, kernel_initializer="uniform", activation = 'sigmoid')
    classifier.add(output_layer)
    classifier.add(Dropout(rate = 0.1)) #disable 10% of the neurons at every iteration 
    
    #compiling ANN
    #set up with stochastic gradient descent, in the case of keras let's use adam optimizer
    #cost function (loss): Logarithmic loss function
    #metrics to evaluate the model
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

## Fitting and Tunning the ANN to the training set 

In [10]:
#Implement K fold cross validation

# Grid search using K cross validation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#create hyperparameters to evaluate
hyperparams = {'batch_size': [32], 
              'epochs': [500], 
              'optimizer': ['adam', 'rmsprop'],
               'nn_count': [6, 8, 10]
             }

#k = 10 folds
#n_jobs = number of cpu's to do the job (-1 to use all cpu's in parallel)
model = KerasClassifier(build_fn=build_classifier)
grid_search = GridSearchCV(estimator = model, param_grid = hyperparams, scoring= 'accuracy', cv = 10)

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/500
7200/7200 [==============================] - 1s 95us/step - loss: 0.9306 - accuracy: 0.4808
Epoch 2/500
7200/7200 [==============================] - 0s 59us/step - loss: 0.7321 - accuracy: 0.7918
Epoch 3/500
7200/7200 [==============================] - 0s 58us/step - loss: 0.6810 - accuracy: 0.7971
Epoch 4/500
7200/7200 [==============================] - 0s 60us/step - loss: 0.7039 - accuracy: 0.7971
Epoch 5/500
7200/7200 [==============================] - 0s 54us/step - loss: 0.7086 - accuracy: 0.7971
Epoch 6/500
7200/7200 [==============================] - 0s 57us/step - loss: 0.7529 - accuracy: 0.7971
Epoch 7/500
7200/7200 [==============================] - 1s 94us/step - loss: 0.7143 - accuracy: 0.7971
Epoch 8/500
7200/7200 [==============================] - 1s 109us/step - loss: 0.7355 - accuracy: 0.7971
Epoch 9/500
7200/7200 [==============================] - 1s 104us/step - loss: 0.7266 - accuracy: 0.7971
Epoch 10/500
7200/7200 [==============================] - 0s 6

7200/7200 [==============================] - 0s 54us/step - loss: 0.7315 - accuracy: 0.7971
Epoch 79/500
7200/7200 [==============================] - 0s 48us/step - loss: 0.7094 - accuracy: 0.7971
Epoch 80/500
7200/7200 [==============================] - 0s 56us/step - loss: 0.7265 - accuracy: 0.7971
Epoch 81/500
7200/7200 [==============================] - 0s 58us/step - loss: 0.7501 - accuracy: 0.7971
Epoch 82/500
7200/7200 [==============================] - 0s 55us/step - loss: 0.6793 - accuracy: 0.7971
Epoch 83/500
7200/7200 [==============================] - 0s 54us/step - loss: 0.7442 - accuracy: 0.7971
Epoch 84/500
7200/7200 [==============================] - 0s 56us/step - loss: 0.6942 - accuracy: 0.7971
Epoch 85/500
7200/7200 [==============================] - 0s 58us/step - loss: 0.6946 - accuracy: 0.7971
Epoch 86/500
7200/7200 [==============================] - 0s 57us/step - loss: 0.7008 - accuracy: 0.7971
Epoch 87/500
7200/7200 [==============================] - 0s 57us/st

7200/7200 [==============================] - 0s 50us/step - loss: 0.6871 - accuracy: 0.7971
Epoch 156/500
7200/7200 [==============================] - 0s 56us/step - loss: 0.7205 - accuracy: 0.7971
Epoch 157/500
7200/7200 [==============================] - 0s 54us/step - loss: 0.7058 - accuracy: 0.7971
Epoch 158/500
7200/7200 [==============================] - 0s 56us/step - loss: 0.6942 - accuracy: 0.7971
Epoch 159/500
7200/7200 [==============================] - 0s 54us/step - loss: 0.7170 - accuracy: 0.7971
Epoch 160/500
7200/7200 [==============================] - 0s 64us/step - loss: 0.6695 - accuracy: 0.7971
Epoch 161/500
7200/7200 [==============================] - 0s 62us/step - loss: 0.7179 - accuracy: 0.7971
Epoch 162/500
7200/7200 [==============================] - 0s 64us/step - loss: 0.7319 - accuracy: 0.7971
Epoch 163/500
7200/7200 [==============================] - 0s 62us/step - loss: 0.7172 - accuracy: 0.7971
Epoch 164/500
7200/7200 [==============================] - 0

7200/7200 [==============================] - 0s 63us/step - loss: 0.6915 - accuracy: 0.7971
Epoch 233/500
7200/7200 [==============================] - 0s 63us/step - loss: 0.7011 - accuracy: 0.7971
Epoch 234/500
7200/7200 [==============================] - 0s 63us/step - loss: 0.7033 - accuracy: 0.7971
Epoch 235/500
7200/7200 [==============================] - 0s 63us/step - loss: 0.7092 - accuracy: 0.7971
Epoch 236/500
7200/7200 [==============================] - 0s 64us/step - loss: 0.6581 - accuracy: 0.7971
Epoch 237/500
7200/7200 [==============================] - 0s 64us/step - loss: 0.6687 - accuracy: 0.7971
Epoch 238/500
7200/7200 [==============================] - 0s 64us/step - loss: 0.6728 - accuracy: 0.7971
Epoch 239/500
7200/7200 [==============================] - 0s 63us/step - loss: 0.6959 - accuracy: 0.7971
Epoch 240/500
7200/7200 [==============================] - 1s 70us/step - loss: 0.7306 - accuracy: 0.7971
Epoch 241/500
7200/7200 [==============================] - 0

7200/7200 [==============================] - 1s 71us/step - loss: 0.7012 - accuracy: 0.7971
Epoch 310/500
7200/7200 [==============================] - 1s 81us/step - loss: 0.6884 - accuracy: 0.7971
Epoch 311/500
7200/7200 [==============================] - 1s 79us/step - loss: 0.6947 - accuracy: 0.7971
Epoch 312/500
7200/7200 [==============================] - 1s 79us/step - loss: 0.7097 - accuracy: 0.7971
Epoch 313/500
7200/7200 [==============================] - 1s 77us/step - loss: 0.7204 - accuracy: 0.7971
Epoch 314/500
7200/7200 [==============================] - 1s 83us/step - loss: 0.6698 - accuracy: 0.8004
Epoch 315/500
7200/7200 [==============================] - 1s 71us/step - loss: 0.7422 - accuracy: 0.7985 0s - loss: 0.7596 - ac
Epoch 316/500
7200/7200 [==============================] - 0s 66us/step - loss: 0.6975 - accuracy: 0.7971
Epoch 317/500
7200/7200 [==============================] - 1s 75us/step - loss: 0.7700 - accuracy: 0.7978 0s - loss: 0.7972 - accuracy
Epoch 31

...

Epoch 159/500
8000/8000 [==============================] - 0s 62us/step - loss: 0.6825 - accuracy: 0.8301
Epoch 160/500
8000/8000 [==============================] - 1s 66us/step - loss: 0.6598 - accuracy: 0.8310
Epoch 161/500
8000/8000 [==============================] - 1s 118us/step - loss: 0.6969 - accuracy: 0.8322
Epoch 162/500
8000/8000 [==============================] - 1s 65us/step - loss: 0.6684 - accuracy: 0.8303
Epoch 163/500
8000/8000 [==============================] - 1s 65us/step - loss: 0.7053 - accuracy: 0.8300
Epoch 164/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.7016 - accuracy: 0.8286
Epoch 165/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.7268 - accuracy: 0.8294
Epoch 166/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.6880 - accuracy: 0.8291
Epoch 167/500
8000/8000 [==============================] - 1s 66us/step - loss: 0.7089 - accuracy: 0.8311
Epoch 168/500
8000/8000 [====================

8000/8000 [==============================] - 1s 77us/step - loss: 0.6687 - accuracy: 0.8313
Epoch 237/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.6905 - accuracy: 0.8332
Epoch 238/500
8000/8000 [==============================] - 1s 78us/step - loss: 0.6989 - accuracy: 0.8288
Epoch 239/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.6866 - accuracy: 0.8299
Epoch 240/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.7072 - accuracy: 0.8300
Epoch 241/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.6939 - accuracy: 0.8329
Epoch 242/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.6731 - accuracy: 0.8313
Epoch 243/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.6678 - accuracy: 0.8286
Epoch 244/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.6666 - accuracy: 0.8299
Epoch 245/500
8000/8000 [==============================] - 1

8000/8000 [==============================] - 1s 80us/step - loss: 0.6693 - accuracy: 0.8313
Epoch 314/500
8000/8000 [==============================] - 1s 105us/step - loss: 0.6536 - accuracy: 0.8321
Epoch 315/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.7024 - accuracy: 0.8306
Epoch 316/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.6960 - accuracy: 0.8295
Epoch 317/500
8000/8000 [==============================] - 1s 79us/step - loss: 0.6718 - accuracy: 0.8311
Epoch 318/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.6794 - accuracy: 0.8300
Epoch 319/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.7061 - accuracy: 0.8300
Epoch 320/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.7008 - accuracy: 0.8290
Epoch 321/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.6879 - accuracy: 0.8294
Epoch 322/500
8000/8000 [==============================] - 

8000/8000 [==============================] - 2s 271us/step - loss: 0.6814 - accuracy: 0.8300
Epoch 388/500
8000/8000 [==============================] - 2s 195us/step - loss: 0.7101 - accuracy: 0.8278
Epoch 389/500
8000/8000 [==============================] - 2s 272us/step - loss: 0.6705 - accuracy: 0.8306
Epoch 390/500
8000/8000 [==============================] - 2s 298us/step - loss: 0.6832 - accuracy: 0.8307
Epoch 391/500
8000/8000 [==============================] - 2s 269us/step - loss: 0.7430 - accuracy: 0.8307
Epoch 392/500
8000/8000 [==============================] - 3s 316us/step - loss: 0.6647 - accuracy: 0.8298
Epoch 393/500
8000/8000 [==============================] - 2s 239us/step - loss: 0.6435 - accuracy: 0.8309
Epoch 394/500
8000/8000 [==============================] - 2s 228us/step - loss: 0.6952 - accuracy: 0.8296
Epoch 395/500
8000/8000 [==============================] - 2s 190us/step - loss: 0.6772 - accuracy: 0.8307
Epoch 396/500
8000/8000 [==========================

8000/8000 [==============================] - 1s 81us/step - loss: 0.6527 - accuracy: 0.8305
Epoch 464/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.6962 - accuracy: 0.8319
Epoch 465/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.6997 - accuracy: 0.8300
Epoch 466/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.6689 - accuracy: 0.8334
Epoch 467/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.7028 - accuracy: 0.8299
Epoch 468/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.6700 - accuracy: 0.8292
Epoch 469/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.6722 - accuracy: 0.8336
Epoch 470/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.6494 - accuracy: 0.8304
Epoch 471/500
8000/8000 [==============================] - 1s 78us/step - loss: 0.6629 - accuracy: 0.8300
Epoch 472/500
8000/8000 [==============================] - 1

In [11]:
best_parameters = grid_search.best_params_
best_parameters

{'batch_size': 32, 'epochs': 500, 'nn_count': 10, 'optimizer': 'adam'}

In [12]:
best_accuracy = grid_search.best_score_
best_accuracy

0.841375

## Evaluate Model

In [13]:
classifier = build_classifier(best_parameters['optimizer'], best_parameters['nn_count'])
classifier.fit(X_train, y_train, batch_size = best_parameters['batch_size'], epochs = best_parameters['epochs'])

Epoch 1/500
8000/8000 [==============================] - 1s 110us/step - loss: 0.8207 - accuracy: 0.6324
Epoch 2/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.7056 - accuracy: 0.7960
Epoch 3/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.7176 - accuracy: 0.7960
Epoch 4/500
8000/8000 [==============================] - 1s 76us/step - loss: 0.7151 - accuracy: 0.7961
Epoch 5/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.6905 - accuracy: 0.7985
Epoch 6/500
8000/8000 [==============================] - 1s 78us/step - loss: 0.7065 - accuracy: 0.8161
Epoch 7/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.7131 - accuracy: 0.8224
Epoch 8/500
8000/8000 [==============================] - 1s 74us/step - loss: 0.7180 - accuracy: 0.8242
Epoch 9/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.7064 - accuracy: 0.8240
Epoch 10/500
8000/8000 [==============================] - 1s 73

8000/8000 [==============================] - 1s 68us/step - loss: 0.6882 - accuracy: 0.8315
Epoch 79/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.6764 - accuracy: 0.8339
Epoch 80/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.6765 - accuracy: 0.8316
Epoch 81/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.7151 - accuracy: 0.8285
Epoch 82/500
8000/8000 [==============================] - 1s 66us/step - loss: 0.6808 - accuracy: 0.8322
Epoch 83/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.6629 - accuracy: 0.8315
Epoch 84/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.6884 - accuracy: 0.8334
Epoch 85/500
8000/8000 [==============================] - 1s 76us/step - loss: 0.6864 - accuracy: 0.8274
Epoch 86/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.6801 - accuracy: 0.8328
Epoch 87/500
8000/8000 [==============================] - 1s 70us/st

8000/8000 [==============================] - 1s 73us/step - loss: 0.7056 - accuracy: 0.8284
Epoch 156/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.6996 - accuracy: 0.8292
Epoch 157/500
8000/8000 [==============================] - 1s 83us/step - loss: 0.6743 - accuracy: 0.8305
Epoch 158/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.6592 - accuracy: 0.8314
Epoch 159/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.6643 - accuracy: 0.8290
Epoch 160/500
8000/8000 [==============================] - 1s 76us/step - loss: 0.6983 - accuracy: 0.8306
Epoch 161/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.6796 - accuracy: 0.8259
Epoch 162/500
8000/8000 [==============================] - 1s 80us/step - loss: 0.6937 - accuracy: 0.8305
Epoch 163/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.6695 - accuracy: 0.8325 0s - loss:
Epoch 164/500
8000/8000 [========================

8000/8000 [==============================] - 1s 112us/step - loss: 0.7100 - accuracy: 0.8306
Epoch 232/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.6800 - accuracy: 0.8276
Epoch 233/500
8000/8000 [==============================] - 1s 106us/step - loss: 0.6619 - accuracy: 0.8301
Epoch 234/500
8000/8000 [==============================] - 1s 123us/step - loss: 0.6557 - accuracy: 0.8339
Epoch 235/500
8000/8000 [==============================] - 1s 132us/step - loss: 0.7231 - accuracy: 0.8292
Epoch 236/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.6407 - accuracy: 0.8309
Epoch 237/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.6853 - accuracy: 0.8291
Epoch 238/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.6981 - accuracy: 0.8317
Epoch 239/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.6863 - accuracy: 0.8282
Epoch 240/500
8000/8000 [============================

8000/8000 [==============================] - 1s 179us/step - loss: 0.6704 - accuracy: 0.8304
Epoch 307/500
8000/8000 [==============================] - 2s 246us/step - loss: 0.6916 - accuracy: 0.8279
Epoch 308/500
8000/8000 [==============================] - 2s 290us/step - loss: 0.6494 - accuracy: 0.8309
Epoch 309/500
8000/8000 [==============================] - 3s 395us/step - loss: 0.6776 - accuracy: 0.8338
Epoch 310/500
8000/8000 [==============================] - 2s 299us/step - loss: 0.6746 - accuracy: 0.83061s - loss: 0.6902 - accuracy: 0.82 - ETA: 1s - loss: 0.6940 - accuracy: 0.82 - E
Epoch 311/500
8000/8000 [==============================] - 3s 357us/step - loss: 0.6598 - accuracy: 0.83090s - loss: 0.6679 - ac
Epoch 312/500
8000/8000 [==============================] - 3s 360us/step - loss: 0.6778 - accuracy: 0.83062s - loss: 0.7005  - ETA: 0s - loss: 0.6696 - accu
Epoch 313/500
8000/8000 [==============================] - 3s 374us/step - loss: 0.6728 - accuracy: 0.8305
Epoch 

8000/8000 [==============================] - 2s 222us/step - loss: 0.7017 - accuracy: 0.8298
Epoch 380/500
8000/8000 [==============================] - 2s 256us/step - loss: 0.6939 - accuracy: 0.8309
Epoch 381/500
8000/8000 [==============================] - 2s 221us/step - loss: 0.7090 - accuracy: 0.8311
Epoch 382/500
8000/8000 [==============================] - 2s 222us/step - loss: 0.7053 - accuracy: 0.83040s - loss: 0
Epoch 383/500
8000/8000 [==============================] - 2s 190us/step - loss: 0.7155 - accuracy: 0.8289
Epoch 384/500
8000/8000 [==============================] - 2s 253us/step - loss: 0.6715 - accuracy: 0.8313
Epoch 385/500
8000/8000 [==============================] - ETA: 0s - loss: 0.7073 - accuracy: 0.82 - 2s 238us/step - loss: 0.7060 - accuracy: 0.8301
Epoch 386/500
8000/8000 [==============================] - 2s 196us/step - loss: 0.7102 - accuracy: 0.8290
Epoch 387/500
8000/8000 [==============================] - 2s 210us/step - loss: 0.7194 - accuracy: 0.82

8000/8000 [==============================] - 1s 140us/step - loss: 0.6322 - accuracy: 0.8339
Epoch 454/500
8000/8000 [==============================] - 1s 129us/step - loss: 0.6539 - accuracy: 0.8301
Epoch 455/500
8000/8000 [==============================] - 1s 136us/step - loss: 0.6749 - accuracy: 0.83000s - loss: 0.6787 - accura
Epoch 456/500
8000/8000 [==============================] - 1s 131us/step - loss: 0.7386 - accuracy: 0.8286
Epoch 457/500
8000/8000 [==============================] - 1s 143us/step - loss: 0.6783 - accuracy: 0.8316
Epoch 458/500
8000/8000 [==============================] - 1s 119us/step - loss: 0.6461 - accuracy: 0.8338
Epoch 459/500
8000/8000 [==============================] - 1s 124us/step - loss: 0.6736 - accuracy: 0.8307
Epoch 460/500
8000/8000 [==============================] - 1s 155us/step - loss: 0.6416 - accuracy: 0.8321
Epoch 461/500
8000/8000 [==============================] - 1s 137us/step - loss: 0.7021 - accuracy: 0.8280
Epoch 462/500
8000/8000 [

In [14]:
y_pred = classifier.predict(X_test)
y_pred_binary = (y_pred > 0.5)

## Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_pred_binary, y_test)
cm

array([[1566,  289],
       [  29,  116]], dtype=int64)

In [16]:
accuracy_score(y_pred_binary, y_test)

0.841

## Predict specific test Entity

In [17]:
test_entity = np.array([[600, 1, 40, 3, 60000, 2, 1, 1, 50000, 0, 0]])
scaled_test_entity = sc.transform(test_entity)
entity_pred = classifier.predict(scaled_test_entity)
entity_pred

array([[0.10377847]], dtype=float32)